<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Competitive_DS/blob/week_6/Competitive_DS_6_2_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2.12

У вас датасет текстов постов из социальной сети и вам нужно классифицировать их по 10 темам, к которым они относятся.

Достаем эмбеддинги при помощи языковых моделей

Очень часто в задачах связанными с текстами выстреливает подход с извлечением из текста эмбеддингов и последующего их использования в бустингах или линейных моделях для решения своей задачи.

В рамках данного степа уже достали для каждого текста из датасета эмбеддинги при помощи больших языковых моделей из HuggingFace, а именно использовали 'sberbank-ai/ruBert-base', 'cointegrated/rubert-tiny2', 'DeepPavlov/rubert-base-cased-conversational' и 'sentence-transformers/LaBSE'.

Так что теперь вы можете использовать их, чтобы составить свой невероятный ансамбль и порвать лидерборд.

Задача: получить максимальное качество классификации, метрика - balanced_accuracy_score, классы в тестовой выборке сбалансированны также, как и в обучающей. За каждый процент точности начиная от 70% вы будете получать +1 балл, максимум 10.

In [1]:
!pip install catboost -q
# !pip install lightgbm -q
# !pip install xgboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.3 MB/s eta 0:00:00


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.svm import SVR

class Blender:
    def __init__(self, base_models, meta_model):

        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X_train, y_train, X_hold, y_hold):
        predicts = []

        for model in self.base_models:
          model.fit(X_train, y_train)
          preds = model.predict(X_hold)
          predicts.append(preds)

        predicts = np.array(predicts)
        predicts = predicts.T

        self.meta_model.fit(predicts, y_hold)

    def predict(self, X_test):
        predicts = []

        for model in self.base_models:
            preds = model.predict(X_test)
            predicts.append(preds)

        predicts = np.array(predicts)
        predicts = predicts.T

        preds = self.meta_model.predict(predicts)

        return preds

# base_models = [LinearRegression(), LassoCV(), SVR()]
# X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=42, shuffle=True)
# # # Теперь обучи свой ансамбль и положия предсказания на X_holdout в переменную preds
# blender = Blender(base_models, LinearRegression())

# # X_train_small, X_hold_small, y_train_small, y_hold_small = train_test_split(X_train, y_train, random_state=42, shuffle=True)

# blender.fit(X_train, y_train, X_holdout, y_holdout)
# preds = blender.predict(X_holdout)

In [3]:
import pandas as pd
import numpy as np

# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from sklearn.model_selection import KFold, StratifiedKFold
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import balanced_accuracy_score

## Загрузка данных

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# path_train = '/content/text_classification_train.csv'
# path_test = '/content/text_classification_test.csv'

path_train = '/content/drive/MyDrive/CompDS_solution_6_2_12/text_classification_train (1).csv'
path_test = '/content/drive/MyDrive/CompDS_solution_6_2_12/text_classification_test (1).csv'

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # else:
        #     df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df
     

print('train')
train = import_data(path_train)

print('-' * 80)
print('test')
test = import_data(path_test)

train
Memory usage of dataframe is 149.80 MB
Memory usage after optimization is: 37.54 MB
Decreased by 74.9%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 49.92 MB
Memory usage after optimization is: 12.49 MB
Decreased by 75.0%


In [ ]:
# train.tail(3)

In [7]:
cat_features = ['text']
text_features = ['text']
cat_features_indxs = [1]
targets = ['category']
features2drop = ['car_id', 'deviation_normal_count']

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in text_features]

# print(f'\033[1mcat_features:\033[0m {cat_features}, \033[1mamount\033[0m = {len(cat_features)}')
# print(f'\033[1mnum_features:\033[0m {num_features}, \033[1mamount\033[0m = {len(num_features)}')
# print(f'\033[1mtargets:\033[0m {targets}, \033[1mamount\033[0m = {len(targets)}')

In [8]:
print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}') 

train shape = (7500, 2618),  test shape = (2500, 2617)
train, test is null: (False, False)


In [10]:
train['category'].value_counts(normalize=True)

autosport       0.081067
athletics       0.080400
boardgames      0.079733
esport          0.078800
tennis          0.078667
martial_arts    0.077867
extreme         0.076267
basketball      0.076000
hockey          0.075067
motosport       0.074933
volleyball      0.074667
football        0.074000
winter_sport    0.072533
Name: category, dtype: float64

In [ ]:
# train.describe(include = "all")

In [ ]:
# train.info()

In [ ]:
# train.iloc[1705]

In [ ]:
train = train.fillna(train.median())
test = test.fillna(test.median())

<ipython-input-8-988d0e28ff34>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train = train.fillna(train.median())
<ipython-input-8-988d0e28ff34>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test = test.fillna(test.median())


In [ ]:
# train.drop(1705, axis=0, inplace=True)

In [ ]:
print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}') 

train shape = (7500, 2618),  test shape = (2500, 2617)
train, test is null: (False, False)


In [ ]:
# train['labse_text_feature_763']

In [ ]:
# test.head(3)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 2617 entries, text to labse_text_feature_767
dtypes: float16(2616), object(1)
memory usage: 12.5+ MB


In [11]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["category"]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_le = le.fit_transform(y)

In [12]:
r_s = 42
n_est = 1000

## 1 Обучить какой-нибудь бустинг

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=r_s)

In [ ]:
clf_1 = CatBoostClassifier(#depth=9,
                        #  l2_leaf_reg=30,
                        #  learning_rate=0.009,
                        #  colsample_bylevel=0.15,
                        #  auto_class_weights= 'SqrtBalanced',
                        #  bootstrap_type = 'Bernoulli',
                         eval_metric= 'Accuracy',
                         iterations=500,
                         thread_count=-1,
                         random_seed=42,
                        #  cat_features=cat_features,
                         text_features=text_features
      )


train_dataset = Pool(data=X_train, label=y_train,
                                #  cat_features=cat_features,
                     text_features=['text'])        
eval_dataset = Pool(data=X_test, label=y_test,
                                # cat_features=cat_features,
                    text_features=['text'])
          
clf_1.fit(train_dataset, 
        eval_set=eval_dataset,
        verbose=200,
        early_stopping_rounds=200)


Learning rate set to 0.145843
0:	learn: 0.3796667	test: 0.4100000	best: 0.4100000 (0)	total: 11.7s	remaining: 1h 37m 16s
200:	learn: 0.8688333	test: 0.7506667	best: 0.7526667 (194)	total: 36m 31s	remaining: 54m 20s
400:	learn: 0.9528333	test: 0.7640000	best: 0.7653333 (379)	total: 1h 11m 41s	remaining: 17m 41s
499:	learn: 0.9718333	test: 0.7693333	best: 0.7713333 (487)	total: 1h 29m	remaining: 0us

bestTest = 0.7713333333
bestIteration = 487

Shrink model to first 488 iterations.


In [ ]:
# bestTest = 0.7713333333
# bestIteration = 487

# {'learn': {'Accuracy': 0.9718333333333333, 'MultiClass': 0.29595477740457804},
#  'validation': {'Accuracy': 0.7713333333333333,
#   'MultiClass': 0.7692615717198646}}

In [ ]:
clf_1.best_score_

{'learn': {'Accuracy': 0.9718333333333333, 'MultiClass': 0.29595477740457804},
 'validation': {'Accuracy': 0.7713333333333333,
  'MultiClass': 0.7692615717198646}}

In [ ]:
# {'learn': {'Accuracy': 0.8956666666666667, 'MultiClass': 0.4996621364950369},
#  'validation': {'Accuracy': 0.7526666666666667,
#   'MultiClass': 0.8189016021234333}}

In [ ]:
# clf_1.best_score_

In [ ]:
# {'learn': {'Accuracy': 0.9992484636265, 'MultiClass': 2.3854503522051136},
#  'validation': {'Accuracy': 0.5095346707132814,
#   'MultiClass': 2.5161996834895066}}

In [ ]:
# clf_1.feature_importances_

In [ ]:
# balanced_accuracy_score(y_pred, valid_data[1])


In [ ]:
# посмотрим на важность признаков катбуста
fi = clf_1.get_feature_importance(prettified=True)
fi[0:750]

,Feature Id,Importances
0,text,35.526440
1,labse_text_feature_402,0.401359
2,labse_text_feature_188,0.393379
3,ruBert-base_text_feature_113,0.389428
4,rubert-base-cased-conversational_text_feature_32,0.369931
...,...,...
745,labse_text_feature_172,0.021505
746,labse_text_feature_617,0.021461
747,labse_text_feature_200,0.021185
748,rubert-base-cased-conversational_text_feature_414,0.021158


In [ ]:
# pd.Series.tolist(fi['Feature Id'])

In [ ]:
# fir = fi[fi['Importances'] == 0]

In [ ]:
fi.to_csv('fi.csv')

In [ ]:
# train['labse_text_feature_763']

0      -0.012711
1      -0.022049
2      -0.047028
3      -0.054657
4      -0.035980
          ...   
1700   -0.007351
1701   -0.025421
1702   -0.063721
1703   -0.072632
1704   -0.027267
Name: labse_text_feature_763, Length: 1705, dtype: float16

In [ ]:
preds = clf_1.predict(
    eval_dataset
)

In [ ]:
balanced_accuracy_score(y_true=y_test, y_pred=preds)

0.77094300358762

## Importance

In [ ]:
fi = pd.read_csv('/content/drive/MyDrive/CompDS_solution_6_2_12/fi.csv')

In [ ]:
important = fi[0:800]['Feature Id'].tolist() #+ ['text']
# important

In [ ]:
X_ = X
# [important]

# X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=42)

## 2 Catboost после feature selection

In [ ]:
# Давайте возьмем топ-250 самых встречаемых и посмотрим как изменится точность.
# important_features = important.append('text')

In [ ]:
#  important = important + ['text']

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, max_features=1000)
# values_stemmed = tfidf_vectorizer.fit_transform(X['text'])


# feature_names = tfidf_vectorizer.get_feature_names_out()
# # # Show the Model as a pandas DataFrame
# # feature_names = tfidf_vectorizer.get_feature_names()
# X_vect = pd.DataFrame(values_stemmed.toarray(), columns = feature_names)

In [ ]:
# X_ = X[important]#.join(X_vect)
# # X

In [ ]:
X_ = X[important]

X_train, X_test, y_train, y_test = train_test_split(X_, y_le, test_size=0.2, random_state=7575)


clf_2 = CatBoostClassifier(#depth=9,
                        #  l2_leaf_reg=30,
                        #  learning_rate=0.009,
                        #  colsample_bylevel=0.15,
                        #  auto_class_weights= 'SqrtBalanced',
                        #  bootstrap_type = 'Bernoulli',
                         eval_metric= 'Accuracy',
                        #  loss_function='MultiClass',

                         iterations=100,
                         thread_count=-1,
                         random_seed=7575,
                        #  cat_features=cat_features,
                        #  text_features=text_features
      )


train_dataset = Pool(data=X_train, label=y_train,
                                #  cat_features=cat_features,
                     text_features=['text'])        
eval_dataset = Pool(data=X_test, label=y_test,
                                # cat_features=cat_features,
                    text_features=['text'])
          
clf_2.fit(train_dataset, 
        eval_set=eval_dataset,
        verbose=200,
        early_stopping_rounds=5)


CatBoostError: ignored

In [ ]:
# clf_2.get_all_params()

In [ ]:
clf_2.get_best_score()

{'learn': {'MultiClass': 0.5894813354227671},
 'validation': {'MultiClass': 0.8722180300928741}}

In [ ]:
Learning rate set to 0.206965
0:	learn: 0.3478333	test: 0.3753333	best: 0.3753333 (0)	total: 4.58s	remaining: 15m 10s
199:	learn: 0.8955000	test: 0.7560000	best: 0.7566667 (179)	total: 17m 43s	remaining: 0us

bestTest = 0.7566666667
bestIteration = 179

Shrink model to first 180 iterations.
<catboost.core.CatBoostClassifier at 0x7fd9d52f49d0>

In [ ]:
clf_3 = CatBoostClassifier(depth=2,
                           l2_leaf_reg=41,
                           learning_rate=0.0099,
                           colsample_bylevel=0.32,
                        #  auto_class_weights= 'SqrtBalanced'
                           bootstrap_type = 'MVS',
                           eval_metric= 'Accuracy',
                           iterations=700,
                           thread_count=-1,
                           random_seed=42,
                        #  cat_features=cat_features,
                           text_features=text_features
      )

In [ ]:
clf_2.best_score_

{'learn': {'Accuracy': 0.979, 'MultiClass': 0.25788800937895184},
 'validation': {'Accuracy': 0.772, 'MultiClass': 0.7432952068906412}}

In [ ]:
preds = clf_2.predict(
    eval_dataset
)

In [ ]:
balanced_accuracy_score(y_true=y_test, y_pred=preds)

0.770975331321264

In [ ]:
# clf_2.get_params()

{'iterations': 700,
 'random_seed': 42,
 'eval_metric': 'Accuracy',
 'text_features': ['text']}

In [ ]:
# clf_2.get_all_params()

# {'nan_mode': 'Min',
#  'eval_metric': 'Accuracy',
#  'iterations': 700,
#  'sampling_frequency': 'PerTree',
#  'leaf_estimation_method': 'Newton',
#  'od_pval': 0,
#  'grow_policy': 'SymmetricTree',
#  'penalties_coefficient': 1,
#  'boosting_type': 'Plain',
#  'model_shrink_mode': 'Constant',
#  'feature_border_type': 'GreedyLogSum',
#  'bayesian_matrix_reg': 0.10000000149011612,
#  'eval_fraction': 0,
#  'force_unit_auto_pair_weights': False,
#  'l2_leaf_reg': 3,
#  'random_strength': 1,
#  'od_type': 'Iter',
#  'rsm': 1,
#  'boost_from_average': False,
#  'model_size_reg': 0.5,
#  'pool_metainfo_options': {'tags': {}},
#  'use_best_model': True,
#  'od_wait': 200,
#  'class_names': ['athletics',
#   'autosport',
#   'basketball',
#   'boardgames',
#   'esport',
#   'extreme',
#   'football',
#   'hockey',
#   'martial_arts',
#   'motosport',
#   'tennis',
#   'volleyball',
#   'winter_sport'],
#  'random_seed': 42,
#  'depth': 6,
#  'posterior_sampling': False,
#  'border_count': 254,
#  'bagging_temperature': 1,
#  'classes_count': 0,
#  'auto_class_weights': 'None',
#  'sparse_features_conflict_fraction': 0,
#  'leaf_estimation_backtracking': 'AnyImprovement',
#  'best_model_min_trees': 1,
#  'model_shrink_rate': 0,
#  'min_data_in_leaf': 1,
#  'text_processing': {'dictionaries': [{'start_token_id': '0',
#     'occurrence_lower_bound': '5',
#     'skip_step': '0',
#     'end_of_word_token_policy': 'Insert',
#     'token_level_type': 'Word',
#     'end_of_sentence_token_policy': 'Skip',
#     'gram_order': '2',
#     'max_dictionary_size': '50000',
#     'dictionary_id': 'BiGram'},
#    {'start_token_id': '0',
#     'occurrence_lower_bound': '5',
#     'skip_step': '0',
#     'end_of_word_token_policy': 'Insert',
#     'token_level_type': 'Word',
#     'end_of_sentence_token_policy': 'Skip',
#     'gram_order': '1',
#     'max_dictionary_size': '50000',
#     'dictionary_id': 'Word'}],
#   'feature_processing': {'default': [{'dictionaries_names': ['BiGram', 'Word'],
#      'feature_calcers': ['BoW'],
#      'tokenizers_names': ['Space']},
#     {'dictionaries_names': ['Word'],
#      'feature_calcers': ['NaiveBayes'],
#      'tokenizers_names': ['Space']}]},
#   'tokenizers': [{'number_token': '🔢',
#     'skip_empty': '1',
#     'number_process_policy': 'LeaveAsIs',
#     'tokenizer_id': 'Space',
#     'token_types': ['Number', 'Unknown', 'Word'],
#     'delimiter': ' ',
#     'languages': [],
#     'lemmatizing': '0',
#     'split_by_set': '0',
#     'lowercasing': '0',
#     'subtokens_policy': 'SingleToken',
#     'separator_type': 'ByDelimiter'}]},
#  'loss_function': 'MultiClass',
#  'learning_rate': 0.12825199961662292,
#  'score_function': 'Cosine',
#  'task_type': 'CPU',
#  'leaf_estimation_iterations': 1,
#  'bootstrap_type': 'Bayesian',
#  'max_leaves': 64}

{'nan_mode': 'Min',
 'eval_metric': 'Accuracy',
 'iterations': 700,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'od_pval': 0,
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'od_type': 'Iter',
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'use_best_model': True,
 'od_wait': 200,
 'class_names': ['athletics',
  'autosport',
  'basketball',
  'boardgames',
  'esport',
  'extreme',
  'football',
  'hockey',
  'martial_arts',
  'motosport',
  'tennis',
  'volleyball',
  'winter_sport'],
 'random_seed': 42,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'bagging_temperature': 1,
 'classes_count': 0,
 'auto_class_weights': 'Non

In [ ]:
0,772

In [ ]:
# clf_1 = CatBoostClassifier(depth=9,
#                          l2_leaf_reg=30,
#                          learning_rate=0.009,
#                          colsample_bylevel=0.15,
#                          auto_class_weights= 'SqrtBalanced',
#                          bootstrap_type = 'Bernoulli',
#                          eval_metric= 'Accuracy',
#                          iterations=500,
#                          thread_count=-1,
#                          random_seed=42,
#                          cat_features=cat_features,
#                         #  text_features=['']
#       )

# base_models = [clf_1]
# X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=42, shuffle=True)
# # # Теперь обучи свой ансамбль и положия предсказания на X_holdout в переменную preds
# blender = Blender(base_models, LinearRegression())

## 3 Optuna

In [ ]:
!pip install optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.model_selection import KFold, train_test_split

In [ ]:
from optuna.integration import CatBoostPruningCallback

In [ ]:
X_ = X[important]

In [ ]:
def fit_catboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        'iterations' : 500, # Можно не перебирать, есть Easrly-Stopping
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 30, 50),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.4),
        
        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 8, 11),
        
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", [#"Bayesian", 
                                                                       "Bernoulli", "MVS"]),
        "used_ram_limit": "10gb",
        "eval_metric": "Accuracy", # Тоже стоит заранее определиться
        # 'loss_function': 'MultiClass'
    }

    
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)
        
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        

    clf = CatBoostClassifier(
        **param,
        thread_count=-1,
        random_seed=42,
        text_features=text_features,
        # cat_features=cat_features
    )
    #  Создаем объект callback
    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
        callbacks=[pruning_callback],
    )

    # запускаем процесс прунинга
    pruning_callback.check_pruned()

    y_pred = clf.predict(X_val)
    return clf, y_pred


In [ ]:
def objective(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    # X_train = train[filtered_features].drop(targets, axis=1, errors="ignore")
    X_train = X_
    y_train = train["category"]

    
    scores, models = [], []
    
    for train_idx, valid_idx in kf.split(X_train):
        train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше
        scores.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break
         

    result = np.mean(scores)
    
    if return_models:
        return result, models
    else:
        return result

In [ ]:
# import sklearn

# categorical_transformer =  Pipeline(steps=[
#     ("imputer", SimpleImputer(strategy='Unknown')),
#     ("onehot", OneHotEncoder(handle_unknown="ignore"))])

# data_pred = categorical_transformer.fit_transform(data[categorical_features])

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),)
study.optimize(objective,
               n_trials=70,
               n_jobs = -1,
               show_progress_bar=True,)

[I 2023-05-03 14:47:21,748] A new study created in memory with name: no-name-ba6de9f6-99dd-49a4-8728-244a03f0561f
/usr/local/lib/python3.10/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/70 [00:00<?, ?it/s]

<ipython-input-17-b611c890d061>:38: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "Accuracy")


[I 2023-05-03 15:14:37,261] Trial 0 finished with value: 0.6729684618901242 and parameters: {'learning_rate': 0.009977584957243563, 'l2_leaf_reg': 41, 'colsample_bylevel': 0.3246314921589391, 'auto_class_weights': 'None', 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.6729684618901242.


<ipython-input-17-b611c890d061>:38: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "Accuracy")


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# Number of finished trials: 4
# Best trial:
#   Value: 0.6781804667809868
#   Params: 
#     learning_rate: 0.00687480100525624
#     l2_leaf_reg: 36
#     colsample_bylevel: 0.4973466535210351
#     auto_class_weights: SqrtBalanced
#     depth: 3
#     boosting_type: Plain
#     bootstrap_type: Bayesian
#     bagging_temperature: 0.8829498476117692

In [ ]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 0.5516171260340416, params {'learning_rate': 0.009416690488249425, 'l2_leaf_reg': 31, 'colsample_bylevel': 0.15021196855587388, 'auto_class_weights': 'SqrtBalanced', 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9900666551619314}


In [ ]:
# Best trial: score 0.5516171260340416, params {'learning_rate': 0.009416690488249425, 'l2_leaf_reg': 31, 'colsample_bylevel': 0.15021196855587388, 'auto_class_weights': 'SqrtBalanced', 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9900666551619314}

In [ ]:
valid_scores, models = objective(
    optuna.trial.FixedTrial(study.best_params),
    return_models=True,
)

In [ ]:
trials_df = study.trials_dataframe().sort_values('value', ascending=False)
trials_df.head(3)

,number,value,datetime_start,datetime_complete,duration,params_auto_class_weights,params_bagging_temperature,params_boosting_type,params_bootstrap_type,params_colsample_bylevel,params_depth,params_l2_leaf_reg,params_learning_rate,params_subsample,state
22,22,0.551617,2023-04-24 12:22:45.828076,2023-04-24 12:50:12.698103,0 days 00:27:26.870027,SqrtBalanced,NaN,Plain,Bernoulli,0.150212,9,31,0.009417,0.990067,COMPLETE
28,28,0.520357,2023-04-24 12:52:49.081447,2023-04-24 13:02:31.995987,0 days 00:09:42.914540,SqrtBalanced,NaN,Plain,Bernoulli,0.141506,8,37,0.009406,0.665457,COMPLETE
15,15,0.519103,2023-04-24 10:56:12.978458,2023-04-24 11:48:39.859980,0 days 00:52:26.881522,SqrtBalanced,NaN,Plain,MVS,0.488303,9,41,0.008496,NaN,COMPLETE


In [ ]:
# История изменения от числа испытаний
optuna.visualization.plot_optimization_history(study)

In [ ]:
valid_scores, len(models)

In [ ]:
# Зависимость в разрере по параметрам
params = ['l2_leaf_reg', 'colsample_bylevel', 
          #'bagging_temperature',
          'depth', 'bootstrap_type', 'subsample', 'learning_rate', 'auto_class_weights']
optuna.visualization.plot_slice(study,
                                params=params,
                                target_name = 'accuracy_score')

In [ ]:
# Важность параметров
optuna.visualization.plot_param_importances(study)

## 4 Обучение после optuna

In [ ]:
# 'learning_rate': 0.009977584957243563, 'l2_leaf_reg': 41, 'colsample_bylevel': 0.3246314921589391, 'auto_class_weights': 'None', 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'

In [ ]:
X_ = X[important]

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=42)


clf_3 = CatBoostClassifier(depth=8,
                           l2_leaf_reg=41,
                          #  learning_rate=0.0099,
                           colsample_bylevel=0.32,
                        #  auto_class_weights= 'SqrtBalanced'
                         bootstrap_type = 'MVS',
                         eval_metric= 'Accuracy',
                         iterations=700,
                         thread_count=-1,
                         random_seed=42,
                        #  cat_features=cat_features,
                         text_features=text_features
      )


train_dataset = Pool(data=X_train, label=y_train,
                                #  cat_features=cat_features,
                     text_features=['text'])        
eval_dataset = Pool(data=X_test, label=y_test,
                                # cat_features=cat_features,
                    text_features=['text'])
          
clf_3.fit(train_dataset, 
        eval_set=eval_dataset,
        verbose=200,
        early_stopping_rounds=200)


0:	learn: 0.3870000	test: 0.4153333	best: 0.4153333 (0)	total: 16.3s	remaining: 3h 9m 21s


KeyboardInterrupt: ignored

In [ ]:
preds_3 = clf_3.predict(
    eval_dataset
)

In [ ]:
balanced_accuracy_score(y_true=y_test, y_pred=preds_3)

## 5 TfIdf

In [14]:
import sys

In [15]:
import joblib

sys.modules['sklearn.externals.joblib'] = joblib

In [ ]:
X_ = X[important]

X_train, X_test, y_train, y_test = train_test_split(X_, y_le, test_size=0.2, random_state=42)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install pymorphy2 -q

In [ ]:
import pymorphy2

In [ ]:
stop_words = set(stopwords.words('russian'))

def text_prepare(text):
    lemmatizer = pymorphy2.MorphAnalyzer()
    word_tokens = word_tokenize(text)
    word_tokens = [w for w in word_tokens if not w in stop_words]
    word_tokens = [lemmatizer.parse(w)[0].normal_form for w in word_tokens]

    return word_tokens

### Bernoulli

In [ ]:
class SelectColumnsNormalizeTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **transform_params):
        corpus_lst = []
        corpus = X[self.columns].values
        lemmatizer = pymorphy2.MorphAnalyzer()
        for text in corpus:
          word_tokens = word_tokenize(text)
          word_tokens = [w for w in word_tokens if not w in stop_words]
          word_tokens = [lemmatizer.parse(w)[0].normal_form for w in word_tokens]
          filtered_text = ' '.join(word_tokens)
          corpus_lst.append(filtered_text)
          
        corpus_lst = np.array(corpus_lst, dtype='object')
        return corpus_lst
        # return corpus

pipe_tf_idf = Pipeline([
    ('col_selector', SelectColumnsNormalizeTransformer('text')),
    ('tfidf', TfidfVectorizer())
    ])

In [ ]:
# X_norm = pipe_tf_idf.fit_transform(X_)
# X_norm


bernoulli_model = BernoulliNB()
B = make_pipeline(pipe_tf_idf, bernoulli_model)

B.fit(X_, y_le)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('col_selector',
                                  <__main__.SelectColumnsNormalizeTransformer object at 0x7fd9d07fc7c0>),
                                 ('tfidf', TfidfVectorizer())])),
                ('bernoullinb', BernoulliNB())])

In [ ]:
preds_tf = B.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_tf)

0.7144624432709551

In [ ]:
class NormalizeText():
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        text = X[self.columns]
        lemmatizer = pymorphy2.MorphAnalyzer()
        word_tokens = word_tokenize(text)
        word_tokens = [w for w in word_tokens if not w in stop_words]
        word_tokens = [lemmatizer.parse(w)[0].normal_form for w in word_tokens]
        return text

In [ ]:
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB





Bernoulli_pipe = Pipeline([
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ('tfidf', TfidfVectorizer()),
    # # ('scaler', StandardScaler()),
    ('bernoulli', BernoulliNB()),
])


Bernoulli_pipe.fit(X_, y_le)

Pipeline(steps=[('col_selector', ColumnSelector(cols='text', drop_axis=True)),
                ('tfidf', TfidfVectorizer()), ('bernoulli', BernoulliNB())])

In [ ]:
preds_tf = Bernoulli_pipe.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_tf)

0.6551336802851433

### SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

SVC_pipe = Pipeline([
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ('tfidf', TfidfVectorizer()),
    ('SVC', SVC()),
])


SVC_pipe.fit(X_, y)

Pipeline(steps=[('col_selector', ColumnSelector(cols='text', drop_axis=True)),
                ('tfidf', TfidfVectorizer()), ('SVC', SVC())])

In [ ]:
preds_tf

array(['autosport', 'volleyball', 'boardgames', ..., 'autosport',
       'autosport', 'basketball'], dtype=object)

In [ ]:
preds_tf = SVC_pipe.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_tf)

ValueError: ignored

### XGBoost

In [ ]:
X_ = X[important]

X_train, X_test, y_train, y_test = train_test_split(X_, y_le, test_size=0.2, random_state=7575)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

XGB_pipe = Pipeline([
    ('col_selector', ColumnSelector(cols=('text'), drop_axis=True)),
    ('tfidf', TfidfVectorizer()),
    # ('XGB', XGBClassifier()),
])


# XGB_pipe.fit(X_, y_le)

In [ ]:
preds_tf = XGB_pipe.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_tf)

0.951150455776074

In [ ]:
# функция для реализации случайного поиска
def random_search_implementation(X, y, algorithm, init_params, grid, fit_params, cv=3, r_s=7575):
    rgr = algorithm(**init_params)
    grid = grid

    model_regr = RandomizedSearchCV(estimator=rgr, 
                                    param_distributions=grid,
                                    cv=cv,
                                    scoring='neg_root_mean_squared_error',
                                    n_jobs=-1,
                                    verbose=100,
                                    n_iter=20,
                                    random_state=r_s)

    model_regr.fit(X, y, **fit_params)

    print('-' * 80)
    print(f'\n\033[1mbest parameters:\033[0m {model_regr.best_params_}')
    print(f'\033[1mbest score =\033[0m {round(abs(model_regr.best_score_), 4)}')
    # print(model_regr.get_params())
    return model_regr.best_params_ | init_params

In [ ]:
X_train = XGB_pipe.fit_transform(X_train)
X_test = XGB_pipe.fit(X_test)

## 6 XGBoost

In [ ]:
X__ = X_.drop('text', axis=1)

In [ ]:
X__.head(3)

,labse_text_feature_402,labse_text_feature_188,ruBert-base_text_feature_113,rubert-base-cased-conversational_text_feature_32,ruBert-base_text_feature_177,ruBert-base_text_feature_641,ruBert-base_text_feature_222,labse_text_feature_548,labse_text_feature_193,labse_text_feature_256,...,rubert-tiny2_text_feature_257,rubert-base-cased-conversational_text_feature_40,rubert-base-cased-conversational_text_feature_365,rubert-tiny2_text_feature_222,ruBert-base_text_feature_513,ruBert-base_text_feature_573,rubert-tiny2_text_feature_280,labse_text_feature_706,ruBert-base_text_feature_169,ruBert-base_text_feature_474
0,-0.012077,0.006767,0.350098,0.122070,0.116760,-0.271729,0.018845,0.019440,-0.021881,-0.042847,...,0.461670,0.441406,-0.329102,-0.399414,0.267822,-0.136108,0.955566,-0.017899,-0.051361,-0.028442
1,0.027847,-0.026871,0.439209,-0.159302,-0.147217,0.104858,0.062378,-0.023300,-0.038635,-0.023773,...,0.393555,0.739746,-0.136475,0.177490,0.134521,0.101196,0.550781,-0.049255,0.169312,-0.169067
2,-0.009071,0.010895,0.599121,0.011757,-0.203003,0.381836,-0.187134,-0.074341,-0.037048,-0.012253,...,0.363281,0.000969,-0.805176,0.125732,-0.086487,0.039734,0.967773,0.011192,-0.198486,0.000805


In [ ]:
xgb_1 = XGBClassifier(enable_categorical=True,
                      # tree_method='hist'
                      )
X_train, X_test, y_train, y_test = train_test_split(X__, y_le, test_size=0.2, random_state=7575)

xgb_1.fit(X_train, y_train)

In [ ]:
preds_st = xgb_1.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_st)

0.6716204371488814

In [ ]:
X__

,labse_text_feature_402,labse_text_feature_188,ruBert-base_text_feature_113,rubert-base-cased-conversational_text_feature_32,ruBert-base_text_feature_177,ruBert-base_text_feature_641,ruBert-base_text_feature_222,labse_text_feature_548,labse_text_feature_193,labse_text_feature_256,...,rubert-tiny2_text_feature_257,rubert-base-cased-conversational_text_feature_40,rubert-base-cased-conversational_text_feature_365,rubert-tiny2_text_feature_222,ruBert-base_text_feature_513,ruBert-base_text_feature_573,rubert-tiny2_text_feature_280,labse_text_feature_706,ruBert-base_text_feature_169,ruBert-base_text_feature_474
0,-0.012077,0.006767,0.350098,0.122070,0.116760,-0.271729,0.018845,0.019440,-0.021881,-0.042847,...,0.461670,0.441406,-0.329102,-0.399414,0.267822,-0.136108,0.955566,-0.017899,-0.051361,-0.028442
1,0.027847,-0.026871,0.439209,-0.159302,-0.147217,0.104858,0.062378,-0.023300,-0.038635,-0.023773,...,0.393555,0.739746,-0.136475,0.177490,0.134521,0.101196,0.550781,-0.049255,0.169312,-0.169067
2,-0.009071,0.010895,0.599121,0.011757,-0.203003,0.381836,-0.187134,-0.074341,-0.037048,-0.012253,...,0.363281,0.000969,-0.805176,0.125732,-0.086487,0.039734,0.967773,0.011192,-0.198486,0.000805
3,0.003860,-0.011887,0.314209,-0.212036,-0.317871,-0.127808,0.028412,-0.056366,-0.034180,-0.048218,...,0.649414,0.437012,-0.788086,-0.277588,0.283936,0.010483,0.822266,0.015717,0.113281,-0.180420
4,0.007660,0.017899,0.373535,-0.073669,-0.410889,-0.018387,0.050262,-0.071167,-0.029648,-0.023651,...,0.413574,0.737793,-0.178833,-0.321777,-0.316895,0.253906,1.044922,-0.000842,-0.022385,0.123962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.041107,-0.051270,0.094421,0.082275,-0.428223,0.180786,-0.025024,-0.034088,-0.046997,0.010307,...,0.322998,0.108704,-0.660645,-0.020630,-0.140991,0.319336,0.210083,-0.041901,0.156860,-0.025238
7496,-0.027954,-0.002119,0.333008,0.141846,-0.245728,0.200195,-0.161621,0.036774,-0.023346,-0.056335,...,0.541504,0.261719,-0.458496,-0.064453,0.072083,0.338135,1.139648,-0.038422,-0.055664,0.045074
7497,-0.064087,-0.032928,0.294189,-0.450928,-0.140869,0.278809,0.051727,0.017929,-0.066467,-0.013443,...,0.130615,-0.012650,-0.506836,-0.155396,0.136597,0.306152,0.783691,-0.001438,0.199951,0.129883
7498,0.001920,-0.058289,0.289307,-0.056488,-0.665527,-0.056702,0.134888,-0.013374,0.026321,0.025528,...,0.481934,0.941895,0.075073,-0.048309,0.237549,0.194458,1.068359,-0.048035,-0.013618,-0.080200


In [ ]:
pipe = Pipeline([
('ct', ColumnTransformer(transformers=[('dropper', 'drop', 'text')],  remainder='passthrough')), 
])
X__ = pipe.fit_transform(X_)
X__

array([[-0.01208  ,  0.006767 ,  0.35     , ..., -0.0179   , -0.05136  ,
        -0.02844  ],
       [ 0.02785  , -0.02687  ,  0.4392   , ..., -0.04926  ,  0.1693   ,
        -0.1691   ],
       [-0.00907  ,  0.010895 ,  0.599    , ...,  0.01119  , -0.1985   ,
         0.0008054],
       ...,
       [-0.0641   , -0.03293  ,  0.2942   , ..., -0.001438 ,  0.2      ,
         0.1299   ],
       [ 0.00192  , -0.0583   ,  0.2893   , ..., -0.04803  , -0.01362  ,
        -0.0802   ],
       [-0.04373  , -0.03555  ,  0.2484   , ..., -0.0662   ,  0.12225  ,
        -0.09     ]], dtype=float16)

## New Stacking

In [16]:
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB

# Вспомогательные блоки организации для пайплайна
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

# Вспомогательные элементы для наполнения пайплайна
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [23]:
stop_words = set(stopwords.words('russian'))

In [18]:
import pymorphy2

In [ ]:
X_ = X
#[important]

X_train, X_test, y_train, y_test = train_test_split(X_, y_le, test_size=0.2, random_state=7575)

In [19]:
class SelectColumnsNormalizeTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **transform_params):
        corpus_lst = []
        corpus = X[self.columns].values
        lemmatizer = pymorphy2.MorphAnalyzer()
        for text in corpus:
          word_tokens = word_tokenize(text)
          word_tokens = [w for w in word_tokens if not w in stop_words]
          word_tokens = [lemmatizer.parse(w)[0].normal_form for w in word_tokens]
          filtered_text = ' '.join(word_tokens)
          corpus_lst.append(filtered_text)
          
        corpus_lst = np.array(corpus_lst, dtype='object')
        return corpus_lst

pipe_tf_idf_true = Pipeline([
    ('col_selector', SelectColumnsNormalizeTransformer('text')),
    ('tfidf', TfidfVectorizer())
    ])

pipe_del_col = Pipeline([('ct', ColumnTransformer(transformers=[('dropper', 'drop', 'text')],  remainder='passthrough'))])

In [20]:
clf_4 = CatBoostClassifier(depth=2,
                           l2_leaf_reg=41,
                           learning_rate=0.0099,
                           colsample_bylevel=0.32,
                        #  auto_class_weights= 'SqrtBalanced'
                           bootstrap_type = 'MVS',
                           eval_metric= 'Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=7575,
                        #  cat_features=cat_features,
                           text_features=text_features
      )

bernoulli_model = BernoulliNB()
svc_model = SVC()
xgb_model = XGBClassifier()
lgbm_model = LGBMClassifier()
xgb_model_2 = XGBClassifier()

In [21]:
# список базовых моделей
estimators = [
    # ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(n_estimators = 10_000, max_depth = 6, min_samples_leaf = 2, 
    #                                                           bootstrap = True, class_weight = 'balanced', # ccp_alpha = 0.001, 
    #                                                           random_state = 75, verbose=False, n_jobs=-1,))),
    

    ("CatBoost", clf_4),
    
    # # То, что не дало прироста в ансамбле
    # # ("SVM", make_pipeline(preprocessor, LinearSVC(verbose=False))),
    # # ("MLP", make_pipeline(preprocessor, MLPClassifier(verbose=False, hidden_layer_sizes=(100, 30, ), alpha=0.001,random_state=75, max_iter = 1300, ))),
    # ("Random_forest",  make_pipeline(preprocessor, RandomForestClassifier(n_estimators = 15_000, max_depth = 7, 
    #                                                           min_samples_leaf = 2,
    #                                                           warm_start = True, n_jobs=-1,
    #                                                           random_state = 75, verbose=False))),
    ('Bernoulli',  make_pipeline(pipe_tf_idf_true, bernoulli_model)),
    ('SVC',  make_pipeline(pipe_tf_idf_true, svc_model)),
    ('XGB',  make_pipeline(pipe_tf_idf_true, xgb_model)),
    ('LGBM', make_pipeline(pipe_tf_idf_true, lgbm_model)),
    ('XGB_2',  make_pipeline(pipe_del_col, xgb_model_2))

    
    
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    # final_estimator=XGBClassifier(),
    # final_estimator=RandomForestClassifier(n_estimators = 10_000, 
                                           # max_depth = 5,
                                           # verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fcb8f5267d0>),
                               ('Bernoulli',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsNormalizeTransformer object at 0x7fcb8f4dfe50>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('bernoullinb',
                                                 BernoulliNB())])),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
stacking_classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fd9d55fb610>),
                               ('Bernoulli',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsNormalizeTransformer object at 0x7fd9d1bd6b30>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('bernoullinb',
                                                 BernoulliNB())])),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
preds_st = stacking_classifier.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_st)

0.8002673454920534

In [24]:
stacking_classifier.fit(X, y_le)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fcb8f5267d0>),
                               ('Bernoulli',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsNormalizeTransformer object at 0x7fcb8f4dfe50>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('bernoullinb',
                                                 BernoulliNB())])),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

## Stacking

In [ ]:
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB

# Вспомогательные блоки организации для пайплайна
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

# Вспомогательные элементы для наполнения пайплайна
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
# preprocessor_tfidf = Pipeline([
#     ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
#     ('tfidf', TfidfVectorizer())])

In [ ]:
# clf_3 = CatBoostClassifier(depth=1,
#                            l2_leaf_reg=41,
#                           #  learning_rate=0.0099,
#                            colsample_bylevel=0.32,
#                         #  auto_class_weights= 'SqrtBalanced'
#                            bootstrap_type = 'MVS',
#                          eval_metric= 'Accuracy',
#                          iterations=700,
#                          thread_count=-1,
#                          random_seed=42,
#                         #  cat_features=cat_features,
#                          text_features=text_features
#       )

# bernoulli_model = BernoulliNB()

# Bernoulli_pipe = Pipeline([
#     ('col_selector', ColumnSelector(cols=('text'), drop_axis=True)),
#     ('tfidf', TfidfVectorizer()),
#     ('bernoulli', bernoulli_model),
# ])




# Bernoulli_pipe.fit(X_, y)

Pipeline(steps=[('col_selector', ColumnSelector(cols='text', drop_axis=True)),
                ('tfidf', TfidfVectorizer()), ('bernoulli', BernoulliNB())])

In [ ]:
# df = sns.load_dataset('tips')
# mycols = ['tip','sex']



# pipeline_select = Pipeline([
#     ("selector", ColumnTransformer([("selector", "passthrough", ['text'])
#     ], remainder="drop"))
# ])

# # pipeline_select.fit_transform(X_)

# bernoulli_model = BernoulliNB()


# bp = make_pipeline(prep_tf_idf,
#                    bernoulli_model)
# bp.fit(X_)

In [ ]:
# pipe = Pipeline([
#     ('col_selector', ColumnSelector(cols=('text'), drop_axis=True)),
# ])

# # pipe.fit_transform(X)

In [ ]:
# class SelectColumnsTransformer():
#     def __init__(self, columns=None):
#         self.columns = columns

#     def fit(self, X, y=None, **fit_params):
#         return self

#     def transform(self, X, **transform_params):
#         # print(X[self.columns].values)
#         cpy_df = X[self.columns].values
#         return cpy_df

    


# # Add it to a pipeline 
# pipe_tf_idf = Pipeline([
#     ('col_selector', SelectColumnsTransformer('text')),
#     ('tfidf', TfidfVectorizer()),
#     # ('bernoulli', BernoulliNB()),
# ])

# # column_select = Pipeline([('col_selector', SelectColumnsTransformer('text'))])

# # tf = Pipeline([('tfidf', TfidfVectorizer())])

# # prep_tf_idf = ColumnTransformer(transformers=[
# #                                               ("col_select", column_select),
# #                                               ("tf", tf)
# #                                               ])



# bernoulli_model = BernoulliNB()


# bp = make_pipeline(pipe_tf_idf,
#                    bernoulli_model)
# bp.fit(X_, y)

# # pipe.fit(X_, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('col_selector',
                                  <__main__.SelectColumnsTransformer object at 0x7f5de1083700>),
                                 ('tfidf', TfidfVectorizer())])),
                ('bernoullinb', BernoulliNB())])

In [ ]:
X_ = X
# [important]

X_train, X_test, y_train, y_test = train_test_split(X_, y_le, test_size=0.2, random_state=7575)

In [ ]:
class SelectColumnsTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **transform_params):
        cpy_df = X[self.columns].values
        return cpy_df

pipe_tf_idf = Pipeline([
    ('col_selector', SelectColumnsTransformer('text')),
    ('tfidf', TfidfVectorizer())
    ])


pipe_del_col = Pipeline([('ct', ColumnTransformer(transformers=[('dropper', 'drop', 'text')],  remainder='passthrough'))])

clf_3 = CatBoostClassifier(depth=2,
                           l2_leaf_reg=41,
                           learning_rate=0.0099,
                           colsample_bylevel=0.32,
                        #  auto_class_weights= 'SqrtBalanced'
                           bootstrap_type = 'MVS',
                           eval_metric= 'Accuracy',
                           iterations=700,
                           thread_count=-1,
                           random_seed=42,
                        #  cat_features=cat_features,
                           text_features=text_features
      )

bernoulli_model = BernoulliNB()
svc_model = SVC()
xgb_model = XGBClassifier()
lgbm_model = LGBMClassifier()
xgb_model_2 = XGBClassifier()

In [ ]:
# список базовых моделей
estimators = [
    
    
    # ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(n_estimators = 10_000, max_depth = 6, min_samples_leaf = 2, 
    #                                                           bootstrap = True, class_weight = 'balanced', # ccp_alpha = 0.001, 
    #                                                           random_state = 75, verbose=False, n_jobs=-1,))),
    

    # ("XGBoost", xgb_model),
    # ("LightGBM", lgbm_model),
    ("CatBoost", clf_3),
    
    # # То, что не дало прироста в ансамбле
    # # ("SVM", make_pipeline(preprocessor, LinearSVC(verbose=False))),
    # # ("MLP", make_pipeline(preprocessor, MLPClassifier(verbose=False, hidden_layer_sizes=(100, 30, ), alpha=0.001,random_state=75, max_iter = 1300, ))),
    # ("Random_forest",  make_pipeline(preprocessor, RandomForestClassifier(n_estimators = 15_000, max_depth = 7, 
    #                                                           min_samples_leaf = 2,
    #                                                           warm_start = True, n_jobs=-1,
    #                                                           random_state = 75, verbose=False))),
    # ('Bernoulli',  make_pipeline(pipe_tf_idf, bernoulli_model)),
    ('SVC',  make_pipeline(pipe_tf_idf, svc_model)),
    ('XGB',  make_pipeline(pipe_tf_idf, xgb_model)),
    ('LGBM', make_pipeline(pipe_tf_idf, lgbm_model)),
    ('XGB_2',  make_pipeline(pipe_del_col, xgb_model_2))

    
    
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    # final_estimator=XGBClassifier(),
    # final_estimator=RandomForestClassifier(n_estimators = 10_000, 
                                           # max_depth = 5,
                                           # verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fa0f26b7340>),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsTransformer object at 0x7fa0f26b6e90>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('svc', SVC())])),
                               ('XGB',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main_...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
stacking_classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fa0f26b7340>),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsTransformer object at 0x7fa0f26b6e90>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('svc', SVC())])),
                               ('XGB',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main_...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [ ]:
preds_st = stacking_classifier.predict(X_test)
balanced_accuracy_score(y_true=y_test, y_pred=preds_st)

0.7790149332775258

In [ ]:
# 0.7622779927974972 - результат 0.7768 7,68
# 0.7790149332775258 - результат 0.7896 8,96 баллов из 10 - обучение на всём датасете.

In [ ]:
# 0.7667985948149748

In [ ]:
# 0.7142578155065581 : depth = 1
# 0.725707327291881 : depth = 6

In [ ]:
stacking_classifier.fit(X_, y_le)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x7fa0f26b7340>),
                               ('SVC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main__.SelectColumnsTransformer object at 0x7fa0f26b6e90>),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('svc', SVC())])),
                               ('XGB',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('col_selector',
                                                                  <__main_...
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

# j,ottt 

In [25]:
x = test[filtered_features].drop(targets, axis=1, errors="ignore")

In [ ]:
le.inverse_transform(stacking_classifier.predict(x))

array(['autosport', 'extreme', 'esport', ..., 'basketball', 'autosport',
       'esport'], dtype=object)

In [26]:
x['category'] = le.inverse_transform(stacking_classifier.predict(x))

In [27]:
predictions = x['category']
predictions

0          extreme
1          extreme
2           esport
3        athletics
4        autosport
           ...    
2495    boardgames
2496        esport
2497    basketball
2498     autosport
2499        esport
Name: category, Length: 2500, dtype: object

In [28]:
predictions.to_csv('try_5.csv', index=False)

In [29]:
import pickle

with open(f'stacking_classifier.pkl', "wb") as f:
        pickle.dump(stacking_classifier, f)

## Rotten

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X['text'] = X['text'].astype('object')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=r_s)

In [ ]:
def fit_catboost_on_rotten_tomatoes(X_train, X_test, y_train, y_test, catboost_params={}, verbose=100):
    learn_pool = Pool(
        X_train, 
        y_train, 
        # cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        # cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    
    catboost_default_params = {
        'iterations': 400,
        'learning_rate': 0.03,
        'eval_metric': 'Accuracy',
    }
    
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)


In [ ]:
fit_catboost_on_rotten_tomatoes(X_train, X_test, y_train, y_test)

0:	learn: 0.2676667	test: 0.2793333	best: 0.2793333 (0)	total: 10.2s	remaining: 1h 7m 32s
100:	learn: 0.6518333	test: 0.7166667	best: 0.7173333 (96)	total: 16m 59s	remaining: 50m 17s
200:	learn: 0.6881667	test: 0.7280000	best: 0.7280000 (194)	total: 33m 32s	remaining: 33m 12s
300:	learn: 0.7338333	test: 0.7380000	best: 0.7380000 (288)	total: 50m 3s	remaining: 16m 27s
399:	learn: 0.7660000	test: 0.7486667	best: 0.7486667 (399)	total: 1h 6m 15s	remaining: 0us

bestTest = 0.7486666667
bestIteration = 399



In [ ]:
import pickle

with open(f'model.pkl', "wb") as f:
        pickle.dump(stacking_classifier, f)

NameError: ignored

##Kfold

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["category"]

n_splits = 3
clfs = []
# scores = []
acc_scores = []

kf = KFold(n_splits=n_splits, shuffle=True, random_state=7575)
for num, (train_index, test_index) in enumerate(kf.split(X)):

  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  clf = CatBoostClassifier(
      cat_features=cat_features
      )
  clfs.append(clf)

  clf.fit(X_train, y_train, 
          eval_set=(X_test, y_test),
          verbose=False, plot=False)
   

  y_pred = clf.predict(X_test)  
  
  acc_score = balanced_accuracy_score(y_test.values, y_pred)

  print(f'fold: {num} acc: {acc_score}')
  # score = np.mean(np.array(y_pred == y_test.values))
  # scores.append(score)
  # print(f"fold: {num} acc: {score}")
  acc_scores.append(acc_score)

assert len(clfs) == n_splits  # Проверка, что все ок

# Считаем среднее и дисперсию по всем фолдам
print("mean accuracy score --", np.mean(acc_score, dtype="float16"), np.std(acc_score).round(4))    

fold: 0 acc: 0.5073460946716484
fold: 1 acc: 0.4729106300938881
fold: 2 acc: 0.450302695042514
mean accuracy score -- 0.4502 0.0


In [ ]:

import pickle

with open(f'model.pkl', "wb") as f:
        pickle.dump(clf, f)